# Install the package.

In [ ]:
!git clone https://github.com/gibsonlab/chronostrain
%pip install chronostrain/.

# Colab-specific installation commands
%pip install click==8.1.3 --force-reinstall
!sudo apt -y install bwa

# Set up the test environment.

In [ ]:
%cp -r chronostrain/examples/colab_demo/configs ./configs
!unzip chronostrain/examples/colab_demo/inputs.zip -d ./inputs

# Example usage

In [ ]:
!chronostrain -h

Use the -c option to pass in a configuration file, along with a command.

In [ ]:
%env CHRONOSTRAIN_INI=configs/chronostrain.ini
%env ENTREZ_EMAIL=
%env CHRONOSTRAIN_LOG_INI=chronostrain/examples/example_configs/log_config.ini

!chronostrain filter -r ./inputs/test_reads.csv -o ./output/filtered --aligner bwa

In [ ]:
!chronostrain advi -r output/filtered/filtered_test_reads.csv -o output/inference